In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://finance.yahoo.com"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
headlines = soup.find_all('h3', class_='Mb(5px)')

In [4]:
for headline in headlines:
    print(headline.get_text())

In [7]:
url = "https://news.yahoo.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
finance_section = soup.find_all('a', href=True)
for link in finance_section:
    if 'finance' in link['href']:
        print(link.get_text(strip=True))

Finance
My portfolio
My watchlist
News
Stock market
Economics
Earnings
Crypto
Politics
Biden economy
Personal finance
Markets
Stocks: most active
Stocks: gainers
Stocks: losers
Trending tickers
Futures
World indices
US Treasury bonds
Currencies
Crypto
Top ETFs
Top mutual funds
Highest open interest
Highest implied volatility
Currency converter
Sectors
Basic materials
Communication services
Consumer cyclical
Consumer defensive
Energy
Financial services
Healthcare
Industrials
Real estate
Technology
Utilities
Screeners
Watchlists
Equities
ETFs
Futures
Index
Mutual funds
Analyst rating screener
Technical events screener
Smart money screener
Top holdings screener
Personal finance
Credit cards
Balance transfer cards
Cash back cards
Rewards cards
Travel cards
Banking
CD rates
Online checking
High-yield savings
Money market
Mortgage
Home equity loan
HELOC
Personal loans
Student loans
Insurance
Taxes
Videos
ETF report
FA corner
Options pit
Crypto
Industries
Finance
Finance
My portfolio
Watchlis

In [31]:
def fetch_article_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.find_all('p')
        text = " ".join([p.get_text() for p in content])
        return text
    except Exception as e:
        print(f"Failed to retrieve or parse article at {url}: {e}")
        return ""

def analyze_article_content(content):
    analysis = TextBlob(content)
    sentiment = analysis.sentiment.polarity
    significant_keywords = [
        'product', 'launch', 'innovation', 'feature', 'upgrade', 'breakthrough', 'patent',
        'earnings', 'revenue', 'profit', 'loss', 'dividend', 'financials', 'forecast', 'guidance',
        'stock', 'share', 'investment', 'valuation', 'market cap', 'performance', 'volatility',
        'regulation', 'lawsuit', 'antitrust', 'compliance', 'investigation',
        'partnership', 'acquisition', 'merger', 'expansion', 'strategic move', 'deal'
    ]
    contains_keywords = any(keyword in content.lower() for keyword in significant_keywords)
    return sentiment, contains_keywords

def fetch_news(company_names):
    api_key = '866cc0d3c22942bd8af6bbf84e2fba11'
    endpoint = "https://newsapi.org/v2/everything"
    query = ' OR '.join(company_names)  # Join company names with OR for the query
    params = {
        'q': query,
        'apiKey': api_key,
        'pageSize': 15,
        'sortBy': 'publishedAt',
    }

    response = requests.get(endpoint, params=params)
    data = response.json()
    if data['status'] == 'ok':
        articles = data['articles']
        results = []
        for article in articles:
            title = article['title']
            url = article['url']
            content = fetch_article_content(url)
            sentiment, contains_keywords = analyze_article_content(content)
            if abs(sentiment) > 0.2 or contains_keywords:
                results.append({
                    "Title": title,
                    "URL": url,
                    "Content": content,
                    "Sentiment": sentiment,
                    "Contains Keywords": contains_keywords
                })
        return results
    else:
        print(f"Failed to fetch news: {data['message']}")
        return []

In [32]:
company_names = ['Apple', 'Microsoft']  # Replace with desired company names
articles_data = fetch_news(company_names)
if articles_data:
    df = pd.DataFrame(articles_data)
    df.to_csv('impactful_articles.csv', index=False)
    print("Data saved to impactful_articles.csv")
else:
    print("No impactful articles found.")

Data saved to impactful_articles.csv


In [33]:
data = pd.read_csv('impactful_articles.csv')

In [34]:
data

,Title,URL,Content,Sentiment,Contains Keywords
0,This is probably the new gold titanium iPhone ...,https://9to5mac.com/2024/08/30/iphone-16-pro-g...,When Apple unveils the iPhone 16 Pro on Septem...,0.166053,True
1,Esperado exclusivo corre por ahora a 25 fps en...,https://www.levelup.com/noticias/801686/Espera...,La secuela llegará a las consolas de Microsof...,-0.106250,True
2,"Save up to $1,100 on three Velotric e-bikes in...",http://9to5toys.com/2024/08/30/save-up-to-1100...,Velotric’s ongoing Summer Sale that is offerin...,0.203506,True
3,Test du Google Pixel 9: Une hausse de prix vra...,https://www.nextpit.fr/google-pixel-9-test-com...,Google a mis à jour le Pixel 9 tout en augment...,-0.213333,True
4,The Secret Spice of ‘Hot Ones’: Host Sean Evan...,https://www.thewrap.com/how-hot-ones-gets-made...,The host tells TheWrap how they snag the bigge...,0.182903,True
5,"ChatGPT tiene 200 millones de usuarios, y un l...",https://www.lanacion.com.ar/tecnologia/chatgpt...,"ChatGPT tiene, a la fecha, 200 millones de usu...",-0.257143,False
6,Legendaria marca de platillos Zildjian se sume...,https://es.digitaltrends.com/vestibles/marca-p...,DT en Español podría recibir una comisión si c...,0.600000,True
7,Neu in macOS Sequoia: Mac App Store nutzt exte...,https://www.ifun.de/neu-in-macos-sequoia-mac-a...,Mit der letzten Vorabversion von macOS Sequoia...,0.656250,True
8,Tecnico assistente informatico,https://lavoro.corriere.it/Annunci/Tecnico_Ass...,Sede di lavoro Categoria - Ruolo Stiamo cercan...,0.437500,True
9,‘Ganymede’ Filmmakers Colby Holt & Sam Probst ...,http://deadline.com/2024/08/colby-holt-sam-pro...,\n\t\t\t\t\tBy Matt Grobar Senior Film Report...,0.129582,True
